# Deploy a Serverless Model Server with Nuclio-KFServing
  --------------------------------------------------------------------

The following notebook demonstrates how to deploy **any pickled model** using **[nuclio](https://github.com/nuclio/nuclio)** + **[KFServing](https://github.com/kubeflow/kfserving)** (a.k.a <b>Nuclio-serving</b>)

#### **notebook how-to's**
* Write and test model serving (KFServing) class in a notebook.
* Deploy the model server as a Nuclio-serving function.
* Invoke and test the serving function.

<a id='top'></a>
#### **steps**
**[define a new function and its dependencies](#define-function)**<br>
**[test the model serving class locally](#test-locally)**<br>
**[deploy our serving class using as a serverless function](#deploy)**<br>
**[test our model server using HTTP request](#test-model-server)**<br>

In [12]:
# nuclio: ignore
# if the nuclio-jupyter package is not installed run !pip install nuclio-jupyter
import nuclio

<a id='define-function'></a>
### **define a new function and its dependencies**

In [13]:
%nuclio config kind="nuclio:serving"
%nuclio env MODEL_CLASS=ClassifierModel

%nuclio: setting kind to 'nuclio:serving'
%nuclio: setting 'MODEL_CLASS' environment variable


In [14]:
%%nuclio cmd -c
pip install -U -q kfserving
pip install -U -q azure
pip install -U -q mlrun

In [15]:
# %nuclio config spec.build.baseImage = "yjbds/mlrun-files:latest"

In [16]:
import kfserving
import os
import numpy as np
from cloudpickle import load

In [17]:
TARGET_PATH = '/User/repos/demos/dask/artifacts'
MODEL_FILE = 'lgbm-model.pkl'

In [18]:
class ClassifierModel(kfserving.KFModel):
    def __init__(self, name: str, model_dir: str, model = None):
        super().__init__(name)
        self.name = name
        self.model_dir = model_dir
        if not model is None:
            self.classifier = model
            self.ready = True

    def load(self):
        model_file = os.path.join(
            kfserving.Storage.download(self.model_dir), MODEL_FILE)
        self.classifier = load(open(model_file, 'rb'))
        self.ready = True

    def predict(self, body):
        try:
            feats = np.asarray(body['instances'])
            result: np.ndarray = self.classifier.predict(feats)
            return result.tolist()
        except Exception as e:
            raise Exception("Failed to predict %s" % e)

The following end-code annotation tells ```nuclio``` to stop parsing the notebook from this cell. _**Please do not remove this cell**_:

In [19]:
# nuclio: end-code

______________________________________________

<a id='test-locally'></a>
### **test the model serving class locally**
The class above can be tested locally. Just instantiate the class, `.load()` will load the model to a local dir.

> **Verify there is a `model.bst` file in the model_dir path (generated by the training notebook)**

In [55]:
model = load(open(TARGET_PATH + '/' + MODEL_FILE, 'rb'))

/User/.pythonlibs/jupyter-1/lib/python3.6/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator LabelEncoder from version 0.21.3 when using version 0.22.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [56]:
model

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.1,
               local_listen_port=12400,
               machines='10.233.64.55:12400,10.233.64.58:12401,10.233.64.56:12402,10.233.64.52:12403,10.233.64.53:12404,10.233.64.54:12405,10.233.64.59:12406,10.233.64.57:12407',
               max_depth=3, min_child_samples=20, min_child_weight=0.001,
               min_split_gain=0.0, n_estimators=3, n_jobs=-1, num_leaves=31,
               num_machines=8, num_threads=1, objective=None, random_state=1,
               reg_alpha=0.0, reg_lambda=0.0, silent=False, subsample=1.0,
               subsample_for_bin=200000, subsample_freq=0, time_out=120,
               tree_learner='data')

In [57]:
my_server = ClassifierModel('classifier', model_dir=TARGET_PATH, model = model)
my_server.load()

[I 200211 15:20:58 storage:35] Copying contents of /User/repos/demos/dask/artifacts to local


### _data_
Make some classification data using scikit learn's `make_classification`:

In [37]:
import dask
import pandas as pd
import dask.dataframe as dd
import pyarrow.parquet as pq
import pyarrow

In [46]:
xtest = pd.read_parquet('/User/repos/demos/dask/artifacts/test_set')

In [47]:
xtest.head()

,index,Year,Month,DayofMonth,DayOfWeek,CRSDepTime,UniqueCarrier,Origin,Dest,Distance,ArrDelay
0,4406,1998,11,11,3,1745.0,0,5,0,859.0,False
1,6183,1998,11,26,4,600.0,0,6,0,259.0,False
2,8481,1998,11,3,2,1300.0,0,9,0,547.0,False
3,1701,1998,11,15,7,700.0,7,27,3,723.0,False
4,25,1998,11,3,2,840.0,2,15,2,328.0,False


In [48]:
xtest.pop('index')

0         4406
1         6183
2         8481
3         1701
4           25
5         3890
6         7234
7         5957
8         2021
9         8363
10         967
11        5455
12        1921
13        5924
14        3595
15        4185
16        7207
17        4415
18        4541
19        1468
20         455
21        1851
22        4401
23        1710
24         285
25        1501
26        5346
27        3365
28        9251
29        9770
          ... 
278534    4479
278535    3094
278536       4
278537    6085
278538    9582
278539    8433
278540    7013
278541     637
278542    9544
278543    8939
278544    5313
278545    3224
278546    6395
278547    4735
278548    8488
278549    5279
278550    3164
278551    5446
278552    2400
278553    2603
278554    7844
278555    1460
278556    7029
278557    2834
278558      14
278559    2753
278560     864
278561    4693
278562    3620
278563    7925
Name: index, Length: 278564, dtype: int64

In [49]:
ytest = xtest.pop('ArrDelay')

In [51]:
event = {"instances": xtest.values.tolist()}

In [53]:
ytest

0         False
1         False
2         False
3         False
4         False
5         False
6         False
7         False
8         False
9         False
10        False
11        False
12        False
13         True
14        False
15         True
16        False
17        False
18         True
19         True
20        False
21        False
22        False
23        False
24        False
25        False
26        False
27        False
28        False
29        False
          ...  
278534     True
278535    False
278536    False
278537    False
278538     True
278539    False
278540    False
278541    False
278542    False
278543    False
278544     True
278545     True
278546     True
278547    False
278548    False
278549     True
278550    False
278551    False
278552    False
278553    False
278554    False
278555     True
278556    False
278557    False
278558    False
278559    False
278560    False
278561    False
278562    False
278563    False
Name: ArrDelay, Length: 

In [52]:
my_server.predict(event)

[False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,


<a id='deploy'></a>
### **deploy our serving class using as a serverless function**

In [60]:
import mlrun
import requests

In [61]:
fn = mlrun.new_model_server('generic', 
                            models={'classifier_gen': TARGET_PATH}, 
                            model_class='ClassifierModel').apply(mlrun.mount_v3io())

In [62]:
fn.spec.no_cache = True

In [63]:
addr = fn.deploy()

[mlrun] 2020-02-11 15:22:03,835 deploy started
[nuclio] 2020-02-11 15:26:04,298 (error) Failed to deploy. Details:

Error - Kaniko job failed. Job logs:
INFO[0000] Resolved base name docker-registry.default-tenant.app.yjb-ds-3.iguazio-cd2.com:80/nuclio/handler-builder-python-onbuild:1.3.13-amd64 to docker-registry.default-tenant.app.yjb-ds-3.iguazio-cd2.com:80/nuclio/handler-builder-python-onbuild:1.3.13-amd64 
INFO[0000] Resolved base name python:3.6 to python:3.6  
INFO[0000] Resolved base name docker-registry.default-tenant.app.yjb-ds-3.iguazio-cd2.com:80/nuclio/handler-builder-python-onbuild:1.3.13-amd64 to docker-registry.default-tenant.app.yjb-ds-3.iguazio-cd2.com:80/nuclio/handler-builder-python-onbuild:1.3.13-amd64 
INFO[0000] Resolved base name python:3.6 to python:3.6  
INFO[0000] Downloading base image docker-registry.default-tenant.app.yjb-ds-3.iguazio-cd2.com:80/nuclio/handler-builder-python-onbuild:1.3.13-amd64 
INFO[0000] Error while retrieving image from cache: getting 

DeployError: cannot deploy 

<a id="test-model-server"></a>
### **test our model server using HTTP request**

In [ ]:
import json
import requests

resp = requests.post(addr + '/classifier_gen/predict', json=event)

In [ ]:
resp.__dict__['_content'] 

In [ ]:
json.loads(resp.content)

**[back to top](#top)**